# KNN+LSH

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
RANDOM_STATE = 42
TRAIN_SIZE = 0.75

In [ ]:
rng = np.random.default_rng(RANDOM_STATE)

## Загрузка данных

In [ ]:
from sklearn.datasets import load_wine

data = load_wine(as_frame=True)

X = data.data
y = data.target

Будем решать задачу мультиклассификации, поэтому посмотрим на количество объектов в каждом классе

In [ ]:
print(y.value_counts())

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = RANDOM_STATE)

Необходимо оценить размерность данных и, если необходимо, масштабировать признаки, ведь того требует KNN

In [ ]:
X_train.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

Обучим KNN с параметрами по умолчанию на тренировочных данных и сделаем предсказание на тесте.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)

Будем измерять качество модели по метрике weighted $f1$-score.

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(y_test, pred, average='weighted'))

Попробуем улучшить модель с помощью `GridSearchCV`

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import numpy as np

params = {'n_neighbors' : np.arange(3, 30, 2), 'weights': ['uniform', 'distance']}

gs = GridSearchCV(KNeighborsClassifier(), params, cv=3, scoring='f1_weighted')


gs.fit(X_train, y_train)

print(gs.best_score_)
print(gs.best_params_)


In [ ]:
clf = KNeighborsClassifier(**gs.best_params_)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
f1_score(y_test, pred, average= 'weighted')

Посмотрим, какие классы между собой путает модель

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred)

## Эксперименты с LSH

Реализуем KNN и LSH с помощью функций
Сначала при помощи LSH отбираем объекты, похожие на объект query
Затем при помощи KNN ищем ближайшие объекты к query только среди похожих, найденных на предыдущем шаге

In [ ]:
def generate_hyperplanes(data, bucket_size=16):
    m = data.shape[0]            # число объектов
    n = data.shape[1]            # число признаков
    b = m // bucket_size         # количество корзин
    h = int(np.log2(b))          # количество гиперплоскостей
    H = rng.normal(size=(h, n))  # гиперплоскости, заданные своими нормалями
    return H

def hamming_hash(data, hyperplanes):
    b = len(hyperplanes)
    hash_key = (data @ hyperplanes.T) >= 0

    dec_vals = np.array([2 ** i for i in range(b)], dtype=int)
    hash_key = hash_key @ dec_vals

    return hash_key
    
def locality_sensitive_hash(data, hyperplanes):
    hash_vals = hamming_hash(data, hyperplanes)
    hash_table = {}
    for i, v in enumerate(hash_vals):
        if v not in hash_table:
            hash_table[v] = set()
        hash_table[v].add(i)

    return hash_table
    
def knn_search(query, data, k=5):

    dists = np.sqrt(np.sum((data - query) ** 2, axis=1))  # вычисляем расстояния от объекта query до всех точек датасета
    inds = np.argsort(dists)  # сортируем по возрастанию расстояний
    inds_k = inds[:k]         # берем top-k точек с наименьшими расстояниями

    return data[inds_k], dists[inds_k]
    
def approx_knn_search(query, data, k=5, bucket_size=16):
    candidates = set()

    hyperplanes = generate_hyperplanes(data)
    hash_table = locality_sensitive_hash(data, hyperplanes) # формируем хеш-таблицу по датасету

    query_hash = hamming_hash(query, hyperplanes)
    if query_hash in hash_table:
        candidates = candidates.union(hash_table[query_hash])
    candidates = np.stack([data[i] for i in candidates], axis=0) # находим кандидатов = объекты, попадающие с query в одну корзину

    return knn_search(query, candidates, k=k) # с помощью KNN ищем ближайших соседей только из найденных кандидатов

При помощи `knn_search` найдем ближайших соседей к вину `X_test.iloc[0]`

In [ ]:
%%time
neighbors, dists = knn_search(X_test.iloc[0].values, X_test.values)
for i, (neighbor, dist) in enumerate(zip(neighbors, dists)):
    print(f"top {i + 1}: dist = {dist}")

Теперь найдем ближайшего соседа при помощи `approx_knn_search`.

In [ ]:
%%time
neighbors, dists = approx_knn_search(X_test.iloc[0].values, X_test.values)
for i, (neighbor, dist) in enumerate(zip(neighbors, dists)):
    print(f"top {i + 1}: dist = {dist}")

Ближайший сосед при помощи KNN+LSH может быть найден не точно или не с первого запуска.  
Необходимо заупстить последнюю ячейку несколько раз и убедиться, что ближайший сосед находится верно за несколько запусков.